<a href="https://colab.research.google.com/github/Chesalov63/Budget_Cars_Forums_NLP/blob/main/1.%20Data%20Extraction/FocusSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib
from urllib.request import urlopen, urlretrieve
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

site = 'https://corolla-club.ru/'
folder_names = [
    ['0', 'Отзывы', 'f=5', 'TOYOTA COROLLA'],
    ['1', 'Двигатель', 'f=8', 'TOYOTA COROLLA'],
    ['2', 'Трансмиссия', '', 'TOYOTA COROLLA'],
    ['3', 'Ходовая', 'f=15', 'TOYOTA COROLLA'],
    ['4', 'Колеса', 'f=11', 'TOYOTA COROLLA'],
    ['5', 'Электрооборудование', 'f=9', 'TOYOTA COROLLA'],
    ['6', 'Мультимедиа', 'f=10', 'TOYOTA COROLLA'],
    ['7', 'Кузов', 'f=7', 'TOYOTA COROLLA'],
    ['8', 'Жидкости', '', 'TOYOTA COROLLA'],
    ['9', 'Сервис', 'f=12', 'TOYOTA COROLLA'],
    ['10', 'Эксплуатация', '', 'TOYOTA COROLLA'],
    ['11', 'Тюнинг', 'f=32', 'TOYOTA COROLLA'],
    ['12', 'Кузов-Вентиляция', 'f=16', 'TOYOTA COROLLA'],
    ['13', 'Сервис-Отзывные компании', 'f=31', 'TOYOTA COROLLA'],
    ['14', 'Тюнинг-Противоугон', 'f=21', 'TOYOTA COROLLA']
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [ ]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [ ]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
       
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}
        
        while flag:
            resp = urlopen(theme_address) # скачиваем файл
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем сами сообщения
            lis = soup.find_all('li')
            for li in lis:
                post_id = ''
                post_data = {}
                if li.has_attr('id') and re.search('post_[0-9]+', li['id']):
                    post_id = re.sub('post_', '', li['id'])
                    ##Собираем данные о дате и авторе сообщения
                    tags = li.find_all('span')
                    for span in tags:
                        if span.has_attr('class') and span['class'] == ['date']:
                            post_data[post_id] = [span.get_text().strip()]
                    tags = li.find_all('a')
                    author_id = '0'
                    author_nickname = 'Guest'
                    for tag in tags:
                        if tag.has_attr('class') and tag['class'][0] == 'username':
                            author_id = re.sub('member.php?u=', '', tag['href'])
                            author_nickname = tag.get_text().strip()
                    post_data[post_id].append([author_id, author_nickname])
                    ##Собираем сами сообщения                            
                    tags = li.find_all('div')
                    for div in tags:
                        if div.has_attr('id') and re.search('post_message_', div['id']):
                            blockquote = div.find('blockquote')
                            additional_divs = blockquote.find_all('div')
                            for a_d in additional_divs:
                                a_d.clear()
                            s = blockquote.get_text().strip()
                            s = re.sub('[\']+', '`', s)
                            s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                            if len(s) > 2:
                                theme_messages.append([post_data[post_id][0], post_data[post_id][1][1], s])

            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
    return messages

In [ ]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [ ]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [ ]:
scraper(site, folder_names)

In [ ]:
#import pandas as pd
#mess = pd.read_csv('./TOYOTA COROLLA/TOYOTA COROLLA_5_Электрооборудование.csv', quotechar="'")
#mess